<a href="https://colab.research.google.com/github/velish-qubadov/Machine-Learning/blob/main/Full_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/refs/heads/main/housing.csv

--2025-02-08 11:15:07--  https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/refs/heads/main/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.07s   

2025-02-08 11:15:07 (20.3 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/housing.csv')

In [ ]:
if df['median_house_value'].isna().sum()>0:
  df.dropna(subset='median_house_value')

In [ ]:
X=df.drop('median_house_value',axis=1)
y=df['median_house_value'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
num_features= X_train.select_dtypes(include=[np.number]).columns
cat_features=X_train.select_dtypes(exclude=[np.number]).columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
num_pipeline= Pipeline([
     ('impute', SimpleImputer(strategy='median')),
     ('scale', StandardScaler())
])
cat_pipeline= Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
transformer= ColumnTransformer([
    ('num', num_pipeline,num_features),
    ('cat', cat_pipeline,cat_features)
],remainder='passthrough')

estimator= RandomForestRegressor()
full_pipeline= Pipeline([
    ('preprocessing', transformer),
    ('estimator', estimator)
])

In [ ]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['ocean_proximity'], dtype='object'))])),
                ('estimator', RandomForestRegressor())])

In [ ]:
full_pipeline.score(X_train, y_train)

0.9747564009116421

In [ ]:
full_pipeline.score(X_test, y_test)

0.8354407517867577

### Train/Validation/Test


In [ ]:
X_train_full,X_test, y_train_full,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train_full,X_test, y_train_full,y_test=train_test_split(X,y,test_size=0.2)

### Hyper-Parametr Tuning



In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV   r

In [ ]:
param_grid={
    'estimator__n_estimators': [100,150,200],
    'estimator__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'estimator__n_jobs' : [-1]
}
grid_scv=GridSearchCV(full_pipeline,param_grid,cv=3)

In [ ]:
grid_scv.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scale',
                                                                                          StandardScaler())]),
                                                                         Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='objec...
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         Index(['ocean_proximity'], dtype='object'))])),
                                       ('estimator', RandomForestRegressor())]),
             param_grid={'estimator__criterion': ['squared_error',
                                                  'absolute_error',
                                                  'friedman_mse', 'poisson'],
                         'estimator__n_estimators': [100, 150, 200],
                         'estimator__n_jobs': [-1]})

In [ ]:
grid_scv.best_params_

In [ ]:
grid_scv.best_estimator_.fit(X_train,y_train)

In [ ]:
best_model=grid_scv.best_params_
best_model.score(X_train,y_train),best_model.score(X_test, y_test)